In [1]:
from paraview.simple import *
from paraview.selection import *

Using a terminal on this node, we request a new allocation:

salloc -N 8 -n 8 -A csstaff -C gpu --reservation tutorial --time=00:20:00

salloc: Granted job allocation 30807331 salloc: Waiting for resource configuration salloc: Nodes nid0[3508-3511] are ready for job

####################

We run pvserver on the newly allocated nodes

srun pvserver

The jupyter client must connect to the first MPI task on the new allocation, i.e. connect to the first node, e.g. nid03508

Connect("nid03508")

In [2]:
Connect("nid02080")

Connection (cs://nid02080:11111) [2]

In [3]:
LoadPlugin("/users/jfavre/Projects/Adamek/ParaViewLightConePlugin/build59/lib64/paraview-5.9/plugins/pvLightConeReader/pvLightConeReader.so", ns=globals())

In [4]:
renderView1 = GetRenderView()

reader = LightConeSeriesReader(FileNames=['/scratch/snx3000/farbodh/Gadget2_snapshot/output_redist/lcdm_Mn0d13_gevolution_snap000_cdm_redist.0'])
reader.PointArrayStatus = ['velocity']
reader.UpdatePipelineInformation()

In [5]:
reader.DistributedSnapshot = 0 # to read a single file (you should be able to run on the single node already allocated)
reader.DistributedSnapshot = 1 # to read all files (you must do a second node allocation

In [6]:
reader.UpdatePipeline()

In [ ]:
mergeBlocks1 = MergeBlocks(Input=reader)
mergeBlocks1.OutputDataSetType = 'Polygonal Mesh'
mergeBlocks1.MergePoints = 0


In [7]:
from ipyparaview.widgets import PVDisplay
disp = PVDisplay(GetActiveView())
w = display(disp)

PVDisplay(resolution=(400, 400))

In [ ]:
repr1 = Show(mergeBlocks1)
repr1.Representation = 'Points'
ColorBy(repr1, ('POINTS', 'velocity', 'Magnitude'))

In [10]:
ResetCamera()


In [ ]:
di = mergeBlocks1.GetDataInformation().DataInformation
print(di.GetNumberOfPoints(), " particles were read\n")

In [8]:
clip1 = Clip(Input=reader)
clip1.ClipType = 'Sphere'
clip1.ClipType.Center = [600000.0, 600000.0, 600000.0]
clip1.ClipType.Radius = 100000.0

In [ ]:
Hide(mergeBlocks1)

In [9]:
repr2 = Show(clip1)

repr2.Representation = 'Points'
ColorBy(repr2, ('POINTS', 'velocity', 'Magnitude'))

In [ ]:
Disconnect()